In [ ]:
pip install gradio py3Dmol


In [ ]:
pip install py3Dmol

In [ ]:
import os
import xml.etree.ElementTree as ET
import py3Dmol

# 👉 STEP 1: Add all your file paths here (absolute or relative)
file_paths = [
    "/content/2yx8.xml",
    "/content/3n7p.xml",
    "/content/3n7r.xml",
    "/content/3n7s.xml",
    "/content/6umg.xml",
    "/content/6umh.xml",
    "/content/6umi.xml",
    "/content/6umj.xml",
    "/content/6zho.xml",
    "/content/6zis.xml",
    "/content/7exd.xml"
]

# 👉 STEP 2: Function to parse XML
def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        ns = {"PDBx": root.tag.split("}")[0][1:]}
        info = {
            "PDB ID": os.path.basename(file_path).split('.')[0],
            "Title": root.findtext(".//PDBx:struct/PDBx:title", default="N/A", namespaces=ns),
            "Release Date": root.findtext(".//PDBx:struct/PDBx:release_date", default="N/A", namespaces=ns)
        }
    except Exception as e:
        info = {"Error": str(e)}
    return info

# 👉 STEP 3: Show 3D structure from PDB
def show_structure(pdb_id):
    view = py3Dmol.view(query='pdb:' + pdb_id)
    view.setStyle({'cartoon': {'color': 'spectrum'}})
    view.zoomTo()
    return view.show()

# 👉 STEP 4: Run on all files
for file_path in file_paths:
    if os.path.exists(file_path):
        info = parse_xml(file_path)
        print("\n" + "="*50)
        for key, value in info.items():
            print(f"{key}: {value}")
        print("="*50)

        # Only show 3D if ID was found
        if "PDB ID" in info:
            show_structure(info["PDB ID"])
    else:
        print(f"File not found: {file_path}")



PDB ID: 2yx8
Title: Crystal structure of the extracellular domain of human RAMP1
Release Date: N/A


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


PDB ID: 3n7p
Title: Crystal structure of the ectodomain complex of the CGRP receptor, a Class-B GPCR, reveals the site of drug antagonism
Release Date: N/A


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


PDB ID: 3n7r
Title: Crystal structure of the ectodomain complex of the CGRP receptor, a Class-B GPCR, reveals the site of drug antagonism
Release Date: N/A


...

3Dmol.js failed to load for some reason. Please check your browser console for error messages.


PDB ID: 6zis
Title: Crystal structure of a CGRP receptor ectodomain heterodimer with bound high affinity inhibitor
Release Date: N/A


3Dmol.js failed to load for some reason. Please check your browser console for error messages.


PDB ID: 7exd
Title: Lasmiditan-bound serotonin 1F (5-HT1F) receptor-Gi protein complex
Release Date: N/A


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
import os
import xml.etree.ElementTree as ET
import py3Dmol
import gradio as gr

# ✅ STEP 1: Add all your .xml file paths here
file_paths = [
    "/content/2yx8.xml",
    "/content/3n7p.xml",
    "/content/3n7r.xml",
    "/content/3n7s.xml",
    "/content/6umg.xml",
    "/content/6umh.xml",
    "/content/6umi.xml",
    "/content/6umj.xml",
    "/content/6zho.xml",
    "/content/6zis.xml",
    "/content/7exd.xml"
]

# ✅ STEP 2: Create a map from PDB ID to XML path
pdb_id_to_file = {
    os.path.basename(path).split('.')[0].upper(): path for path in file_paths if os.path.exists(path)
}
pdb_ids = list(pdb_id_to_file.keys())

# ✅ STEP 3: XML parsing function
def parse_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        ns = {"PDBx": root.tag.split("}")[0][1:]}
        info = {
            "PDB ID": os.path.basename(file_path).split('.')[0],
            "Title": root.findtext(".//PDBx:struct/PDBx:title", default="N/A", namespaces=ns),
            "Release Date": root.findtext(".//PDBx:struct/PDBx:release_date", default="N/A", namespaces=ns)
        }
    except Exception as e:
        info = {"Error": str(e)}
    return info

# ✅ STEP 4: Render structure
def generate_3d_view(pdb_id):
    try:
        view = py3Dmol.view(query='pdb:' + pdb_id)
        view.setStyle({'cartoon': {'color': 'spectrum'}})
        view.zoomTo()
        return view._make_html()
    except Exception as e:
        return f"<p style='color:red'>3D structure error: {e}</p>"

# ✅ STEP 5: Function that runs on dropdown selection
def show_structure(pdb_id):
    if pdb_id not in pdb_id_to_file:
        return "PDB ID not found.", ""

    file_path = pdb_id_to_file[pdb_id]
    info = parse_xml(file_path)
    info_text = "\n".join([f"{k}: {v}" for k, v in info.items()])
    html_3d = generate_3d_view(pdb_id)

    return info_text, html_3d

# ✅ STEP 6: Build Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Migraine Protein Viewer (Local XML Files)")

    dropdown = gr.Dropdown(choices=pdb_ids, label="Select PDB ID")
    info_box = gr.Textbox(label="Protein Info", lines=6)
    view_box = gr.HTML(label="3D Structure Viewer")

    dropdown.change(show_structure, inputs=dropdown, outputs=[info_box, view_box])

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6a0e0bca16b4a93fe4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
